In [12]:
import requests

In [13]:
BASE_URL = "http://77.239.108.6:6543"

In [15]:
# # Отправка ответа "Да" для перехода в следующую фазу (идентификация -> debt_discussion)
response = requests.post(
    f"{BASE_URL}/api/chat",
    json={"chat_id": chat_id, "message": "А как можно оплатить через ваш сайт или мобильное приложение?"},
)
data = response.json()
print("Ответ бота после подтверждения:", data)


NameError: name 'chat_id' is not defined

In [24]:
import requests

BASE_URL = "http://77.239.108.6:6543"

# 1. Создание нового чата с передачей ФИО и компании
response = requests.post(
    f"{BASE_URL}/api/get_new_chat_id",
    json={
        "full_name": "Криворучко Александра Ильинична",
        "company_name": "МосЭнерго"
    }
)
data = response.json()
print("Полученный ответ:", data)
chat_id = data["chat_id"]
print("Получен chat_id:", chat_id)

Полученный ответ: {'chat_id': 'd2b1a7d1-9c9a-4dfb-9a06-445b026c9663', 'message': 'Здравствуйте. Меня зовут цифровой помощник, я представляю компанию МосЭнерго. Звоню по вопросу задолженности по коммунальным услугам. В целях контроля качества разговор записывается. Подскажите, пожалуйста, я сейчас разговариваю c Криворучко Александра Ильинична?'}
Получен chat_id: d2b1a7d1-9c9a-4dfb-9a06-445b026c9663


In [28]:
# 2. Отправка сообщения с динамическими переменными (все значения можно менять)
response = requests.post(
    f"{BASE_URL}/api/chat",
    json={
        "chat_id": chat_id,
        "message": "А сколько дней я предлагал?",
        "full_name": "Криворучко Александра Ильинична",
        "account_number": "555888333",
        "debt_amount": "99 999 руб.",
        "address": "Москва, ул. Победы, д. 1",
        "company_name": "МосЭнерго",
        "company_phone": "8-800-123-45-67",
        "partial_payment_amount": "50 000 руб."
    }
)
data = response.json()
print("Ответ бота:", data)


Ответ бота: {'chat_id': 'd2b1a7d1-9c9a-4dfb-9a06-445b026c9663', 'message': 'Вы упомянули, что планируете оплатить через 5 дней. Однако, чтобы избежать начисления штрафов и других негативных последствий, я рекомендую произвести оплату в течение 3 дней. Вы готовы рассмотреть этот вариант?'}


In [ ]:
from docx import Document
from docx.shared import RGBColor
from docx.oxml import parse_xml
from docx.oxml.ns import nsdecls

doc = Document()
doc.add_heading('Документация API чат-бота взыскания задолженности', 0)

# 1. Создание нового чата
doc.add_heading('1. Создание нового чата', level=1)
doc.add_paragraph('Эндпоинт: POST /api/get_new_chat_id')
doc.add_paragraph('Описание: Создаёт новый чат и возвращает уникальный chat_id и приветственное сообщение. Позволяет персонализировать обращение к клиенту.')

doc.add_heading('Входные параметры:', level=2)
table = doc.add_table(rows=1, cols=3)
# Оранжевая заливка для заголовка
shading_elm = parse_xml(r'<w:shd {} w:fill="FFA500"/>'.format(nsdecls('w')))
for cell in table.rows[0].cells:
    cell._tc.get_or_add_tcPr().append(shading_elm)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Параметр'
hdr_cells[1].text = 'Тип'
hdr_cells[2].text = 'Описание'
params = [
    ('full_name', 'string', 'Имя клиента для приветствия'),
    ('company_name', 'string', 'Название компании'),
]
for p, t, d in params:
    row = table.add_row().cells
    row[0].text = p
    row[1].text = t
    row[2].text = d

doc.add_heading('Пример запроса:', level=2)
doc.add_paragraph(
    '''POST /api/get_new_chat_id
Content-Type: application/json

{
    "full_name": "Степанчук Алена Сергеевна",
    "company_name": "ЭнергоСбыт Плюс"
}'''
)

doc.add_heading('Пример ответа:', level=2)
doc.add_paragraph(
    '''{
    "chat_id": "783c6b4f-6ab2-4314-83e8-660cc0a6c735",
    "message": "Здравствуйте. Меня зовут цифровой помощник, я представляю компанию ЭнергоСбыт Плюс. Звоню по вопросу задолженности по коммунальным услугам. В целях контроля качества разговор записывается. Подскажите, пожалуйста, я сейчас разговариваю c Степанчук Алена Сергеевна?"
}'''
)
doc.add_paragraph('В ответе вы получаете идентификатор чата (chat_id) и стартовую фразу, с которой начинается диалог.')

doc.add_page_break()

# 2. Ведение диалога с передачей динамических переменных
doc.add_heading('2. Ведение диалога с передачей динамических переменных', level=1)
doc.add_paragraph('Эндпоинт: POST /api/chat')
doc.add_paragraph('Описание: Отправляет сообщение в чат и получает ответ от бота. Можно передавать любые переменные для подстановки в промпты. LLM запоминает диалог до завершения сессии. В ответе всегда возвращается статус диалога: "in_progress" — диалог продолжается, "finished" — диалог завершён (бот попрощался).')

doc.add_heading('Входные параметры:', level=2)
table = doc.add_table(rows=1, cols=3)
shading_elm = parse_xml(r'<w:shd {} w:fill="FFA500"/>'.format(nsdecls('w')))
for cell in table.rows[0].cells:
    cell._tc.get_or_add_tcPr().append(shading_elm)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Параметр'
hdr_cells[1].text = 'Тип'
hdr_cells[2].text = 'Описание'
params = [
    ('chat_id', 'string', 'Идентификатор чата'),
    ('message', 'string', 'Сообщение пользователя'),
    ('full_name', 'string', 'Имя клиента для обращения'),
    ('account_number', 'string', 'Лицевой счет'),
    ('debt_amount', 'string', 'Сумма долга'),
    ('address', 'string', 'Адрес'),
    ('company_name', 'string', 'Название компании'),
    ('company_phone', 'string', 'Телефон компании'),
    ('partial_payment_amount', 'string', 'Сумма частичной оплаты'),
]
for p, t, d in params:
    row = table.add_row().cells
    row[0].text = p
    row[1].text = t
    row[2].text = d

doc.add_heading('Пример запроса:', level=2)
doc.add_paragraph(
    '''POST /api/chat
Content-Type: application/json

{
    "chat_id": "783c6b4f-6ab2-4314-83e8-660cc0a6c735",
    "message": "да",
    "full_name": "Степанчук Алена Сергеевна",
    "account_number": "555888333",
    "debt_amount": "50 999 руб.",
    "address": "Москва, ул. Победы, д. 1",
    "company_name": "ЭнергоСбыт Плюс",
    "company_phone": "8-800-123-45-67",
    "partial_payment_amount": "50 000 руб."
}'''
)

doc.add_heading('Пример ответа:', level=2)
doc.add_paragraph(
    '''{
    "chat_id": "783c6b4f-6ab2-4314-83e8-660cc0a6c735",
    "message": "На вашем лицевом счёте 555888333 по адресу Москва, ул. Победы, д. 1 образовалась задолженность в размере 50 999 руб. Скажите, пожалуйста, когда вы планируете произвести оплату?",
    "status": "in_progress"
}'''
)
doc.add_paragraph('В ответе вы получаете ответ бота и статус диалога.')

doc.add_page_break()

# 3. Получение списка всех активных чатов
doc.add_heading('3. Получение списка всех активных чатов', level=1)
doc.add_paragraph('Эндпоинт: GET /api/get_all_chat_ids')
doc.add_paragraph('Описание: Возвращает список всех активных chat_id.')

doc.add_heading('Пример запроса:', level=2)
doc.add_paragraph('GET /api/get_all_chat_ids')

doc.add_heading('Пример ответа:', level=2)
doc.add_paragraph(
    '''[
    "783c6b4f-6ab2-4314-83e8-660cc0a6c735",
    "30e92028-0618-40d3-8afa-92f303159b06"
]'''
)

doc.add_page_break()

# 4. Удаление чата по chat_id
doc.add_heading('4. Удаление чата по chat_id', level=1)
doc.add_paragraph('Эндпоинт: POST /api/remove_chat_by_id')
doc.add_paragraph('Описание: Удаляет чат по его идентификатору.')

doc.add_heading('Входные параметры:', level=2)
table = doc.add_table(rows=2, cols=3)
table.style = 'Light List Accent 1'
shading_elm = parse_xml(r'<w:shd {} w:fill="FFA500"/>'.format(nsdecls('w')))
for cell in table.rows[0].cells:
    cell._tc.get_or_add_tcPr().append(shading_elm)
table.rows[0].cells[0].text = 'Параметр'
table.rows[0].cells[1].text = 'Тип'
table.rows[0].cells[2].text = 'Описание'
table.rows[1].cells[0].text = 'chat_id'
table.rows[1].cells[1].text = 'string'
table.rows[1].cells[2].text = 'Идентификатор чата для удаления'

doc.add_heading('Пример запроса:', level=2)
doc.add_paragraph(
    '''POST /api/remove_chat_by_id
Content-Type: application/json

{
    "chat_id": "783c6b4f-6ab2-4314-83e8-660cc0a6c735"
}'''
)

doc.add_heading('Пример ответа:', level=2)
doc.add_paragraph(
    '''{
    "message": "chat deleted"
}'''
)

doc.save("Документация_API_чат-бота.docx")

In [19]:
import requests

BASE_URL = "http://77.239.108.6:6543"


# 2. Отправка сообщения с динамическими переменными (все значения можно менять)
response = requests.post(
    f"{BASE_URL}/api/chat",
    json={
        "chat_id": chat_id,
        "message": "А кто это звонит?",
        "full_name": "Криворучко Александра Ильинична",
        "account_number": "555888333",
        "debt_amount": "99 999 руб.",
        "address": "Москва, ул. Победы, д. 1",
        "company_name": "МосЭнерго",
        "company_phone": "8-800-123-45-67",
        "partial_payment_amount": "50 000 руб."
    }
)
data = response.json()
print("Ответ бота:", data)


Ответ бота: {'chat_id': 'c8277bc3-c15c-4a35-96f9-364876bf49e8', 'message': 'Я цифровой помощник МосЭнерго. Я разговариваю с Криворучко Александра Ильинична?'}
